In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import utils
import constants
import chess
import chess.pgn
import numpy as np
import os
import random

Data files: 1 - 27
 - each has 1 000 000 rows
 - treat 1 - 25 as training data
 - 26 as final test data
 - 27 as validation data after training is completed (for preliminary comparisons)

In [2]:
def split_pgn():
    pgn_files = [f"data/chessgames20{i}.pgn" for i in range(16, 22)]
    outcomes = {"1":1.0, "1/2":0.5, "0":0.0}
    df = []
    file_counter = 1
    for pgn_file in pgn_files:
        print(f"{pgn_file = }")
        with open(pgn_file, "r") as f:
            game = chess.pgn.read_game(f)
            while game != None:
                result = game.headers["Result"].split("-")
                result = outcomes[result[0]]  # 1 if white wins 0 if black wins 0.5 if stalemate
                board = game.board()
                for move in game.mainline_moves():
                    df.append({"board":board.fen(), "move":constants.uci_moves[move.uci()], "outcome":result})
                    board.push(move)
                if len(df) >= 1000000:
                    df = pd.DataFrame(df)
                    df.to_csv(f"data/split/chess{file_counter}.csv", index=False)
                    file_counter += 1
                    df = []
                game = chess.pgn.read_game(f)

def shuffle_csv(rounds = 10, batch_size=10):
    files = [f"data/split/chess{i}.csv" for i in range(1, 27)]
    n_rows = 1000000
    leftover = pd.DataFrame([], columns=["board", "move", "outcome"])
    for i in range(rounds):
        print(f"round {i+1}/{rounds}")
        picked_files = random.sample(files, batch_size)
        df = pd.DataFrame([], columns=["board", "move", "outcome"])
        for file in picked_files:
            df = pd.concat([df, pd.read_csv(file)], ignore_index=True)
        df = df.sample(frac=1)
        for n, file in enumerate(picked_files):
            df.iloc[(n_rows*n):(n_rows*(n+1)), :].to_csv(file, index=False)
        leftover = pd.concat([leftover, df.iloc[(n_rows*(n+1)):, :]])
    leftover = leftover.sample(frac=1)
    for i in range(len(leftover)//n_rows+1):
        leftover.iloc[(n_rows*i):(n_rows*(i+1))].to_csv(f"data/split/chess{27+i}.csv", index=False)

def process_df(df):
    obs = df["board"].map(utils.parse_fen)
    df["obs_board"] = obs.map(lambda x: x[:64])
    df["obs_misc"] = obs.map(lambda x: x[64:])
    return df

def process_file(file_numbers):
    for n in file_numbers:
        print(f"proccessing file {n}")
        df = pd.read_csv(f"data/split/chess{n}.csv")
        df = process_df(df)
        df.to_pickle(f"data/split/processed/chess{n}.pkl")
# split_pgn()
# shuffle_csv()
# process_file([1, 2, 3, 4, 5])

proccessing file 1
proccessing file 2
proccessing file 3
proccessing file 4
proccessing file 5


In [3]:
def restrict_legal(move_arr, fen):
    ''' Restricts the move probability array to only legal moves and rescale remaining legal move probabilities '''
    board = chess.Board(fen)
    legal_arr = np.zeros(constants.LEN_UCI_MOVES, dtype=np.int16)
    for move in board.legal_moves:
        legal_arr[constants.uci_moves[move.uci()]] = 1
    for i in range(constants.LEN_UCI_MOVES):
        if legal_arr[i] == 0:
            move_arr[i] = 0
    return move_arr / np.sum(move_arr)

In [4]:
def train_model(model, data_dir="data/split/", n_files=25, epochs=1, ckpt_dir="ckpt/actor"):

    file_order = list(range(1, n_files+1))

    if not os.path.exists(ckpt_dir):
        os.mkdir(ckpt_dir)
    if not os.path.exists(utils.path_join(ckpt_dir, model.name)):
        os.mkdir(utils.path_join(ckpt_dir, model.name))

    checkpoints = [utils.path_join(ckpt_dir, model.name, name) for name in os.listdir(utils.path_join(ckpt_dir, model.name))]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        model = keras.models.load_model(latest_checkpoint)
        n = latest_checkpoint.split("-")[-1][0]
    else:
        n = 0

    processed_files = [filename.split(".")[0] for filename in os.listdir(utils.path_join(data_dir, "processed"))]
    for file in file_order:
        if file <= int(n):
            continue
        filename = f"chess{file}"
        print(f"Training on file {filename}")

        # load data from .csv file
        if filename in processed_files:
            df = pd.read_pickle(utils.path_join(data_dir, "processed", filename + ".pkl"))
        else:
            df = pd.read_csv(utils.path_join(data_dir, filename + ".csv"))

            # translate from fen to obs arr
            print("Processing...", end="")
            df = process_df(df)
            print("complete")

        # convert to numpy arrays
        x = {"board": np.array(df["obs_board"].values.tolist()), "misc": np.array(df["obs_misc"].values.tolist())}
        y = df["move"].values

        # train
        model.fit(x, y, batch_size=64, epochs=epochs, validation_split=0.1)

        # save checkpoint between files
        model.save(utils.path_join(ckpt_dir, model.name, f"{model.name}-{file}.h5"))
    print(f"Training completed. Final save file: {utils.path_join(ckpt_dir, model.name, f'{model.name}-{file}.h5')}")

In [7]:
def evaluate_model(model_names, data="validation", data_dir="data/split/", ckpt_dir="ckpt/actor", metric="sparse_categorical_accuracy"):

    models = []
    for model_name in model_names:
        checkpoints = [utils.path_join(ckpt_dir, model_name, name) for name in os.listdir(utils.path_join(ckpt_dir, model_name))]
        if checkpoints:
            latest_checkpoint = max(checkpoints, key=os.path.getctime)
            print("Restoring from", latest_checkpoint)
            model = keras.models.load_model(latest_checkpoint)
            models.append(model)
        else:
            print("No checkpoint available")

    # load data
    if data == "validation":
        df = pd.read_csv(utils.path_join(data_dir, "chess25.csv"))
    elif data == "test":
        df = pd.read_csv(utils.path_join(data_dir, "chess26.csv"))
    
    # convert fen to obs
    print("Processing...", end="")
    df = process_df(df)
    print("complete")

    # convert to numpy arrays
    x = {"board": np.array(df["obs_board"].values.tolist()), "misc": np.array(df["obs_misc"].values.tolist())}
    y = data["move"].values

    performance = []
    for model_name, model in zip(model_names, models):
        performance.append([model_name, model.evaluate(x, y, batch_size=128, return_dict=True)[metric]])

    return pd.DataFrame(performance, columns=["model_name", metric])

Restoring from ckpt/actor/co360_cl5_cs100_d0/co360_cl5_cs100_d0-4.h5
Training on file chess5


2022-09-25 18:51:30.513158: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


14063/14063 [==============================] - 573s 41ms/step - loss: 3.0828 - sparse_categorical_accuracy: 0.2602 - val_loss: 3.0643 - val_sparse_categorical_accuracy: 0.2645
Training completed. Final save file: ckpt/actor/co360_cl5_cs100_d0/co360_cl5_cs100_d0-5.h5


In [8]:
models = ["co256_cl4_d0", "co256_cl5_d0", "co300_cl4_d0", "co256_cl4_cs80_d0", "co360_cl5_cs100_d0"]
results = evaluate_model(models)
results

Restoring from ckpt/actor/co256_cl4_d0/co256_cl4_d0-5.h5
Restoring from ckpt/actor/co256_cl5_d0/co256_cl5_d0-5.h5


/Users/kenngkz/.pyenv/versions/3.10.5/envs/chess-ai/lib/python3.10/site-packages/keras/layers/core/lambda_layer.py:303: UserWarning: tf_models.conv2d_seq is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(config, custom_objects,


Restoring from ckpt/actor/co300_cl4_d0/co300_cl4_d0-5.h5
Restoring from ckpt/actor/co256_cl4_cs80_d0/co256_cl4_cs80_d0-5.h5
Restoring from ckpt/actor/co360_cl5_cs100_d0/co360_cl5_cs100_d0-5.h5
Processing...

KeyboardInterrupt: 